## $\Large{Pandas\; Exercise\; Advanced}$

In [ ]:
import pandas as pd
import numpy as np

---
# Airline dataset

這邊有一份飛機航班的資料，分別記錄了幾個飛機班次的起降時間和機場編號，請大家完成以下任務
1. 去除重複的資料
2. 針對每一列計算此班班機的飛行時間(flight_length)
3. 將資料中飛行時間小於0的列挑選出來

### Tips
    計算飛行時間時，可以先把資料轉為datetime格式再做計算
    飛行時間可用arrive_time - fly_time做計算，並使用pd.Timedelta將單位改為分鐘


In [ ]:
csv_path = 'dataset/airline.csv'

In [ ]:
# 讀取資料
df = pd.read_csv(csv_path, index_col=0)

# 刪除資料重複的欄位
df = df.drop_duplicates()

# 將arrive_time 與 fly_time 轉換為 datetime形式再做相減，最後使用pd.Timedelta將單位以分鐘表示
df['flight_length'] = (pd.to_datetime(df.arrive_time) - pd.to_datetime(df.fly_time)) / pd.Timedelta(minutes=1)

In [ ]:
# 挑選flight_length小於0的資料
df_new = df[df.flight_length < 0]

# 印出前五筆資料
df_new.head()

---
理論上飛行時間不會有負數，然而從上面計算出來的結果會發現有小於0的狀況，這是因為我們還沒有對時區差異進行校正。

1. 下方提供了機場與時區的資料(df_timezone)，請將df與df_timezone依照起飛機場作合併，接著去做時區校正後的飛行時間。

### Tips
    你可以先將起飛與降落機場的時區差異計算出來，再依此去校正飛行時間

In [ ]:
df_timezone = pd.read_csv('dataset/time_zone.csv')

In [ ]:
# 合併資料
merge_df = df.merge(df_timezone, left_on='src_airport', right_on='airport', how='left')

# 先複製一份資料以免更動原始資料
cal_df = df.copy()

# 將起飛機場的時區併入資料
cal_df = cal_df.merge(df_timezone, left_on='src_airport', right_on='airport', how='left').drop('airport', axis=1)
# 更改欄位名稱，註記此欄位為起飛機場的時區
cal_df.rename({'time_zone':'time_zone_src'}, axis=1, inplace=True)

# 同樣針對降落機場時區作一次合併與更名
cal_df = cal_df.merge(df_timezone, left_on='dst_airport', right_on='airport', how='left').drop('airport', axis=1).rename({'time_zone':'time_zone_dst'}, axis=1)

# 計算時區差異，並且改為分鐘數
cal_df['time_zone_diff'] = (cal_df['time_zone_src'] - cal_df['time_zone_dst'])*60

# 校正飛行時間
cal_df['flight_length'] = cal_df['flight_length'] + cal_df['time_zone_diff']

# 刪除中途計算的欄位
cal_df.drop(columns=['time_zone_src', 'time_zone_dst', 'time_zone_diff'], inplace=True)

In [ ]:
cal_df.head()

---
# Ted dataset

此份資料收集了過去幾年Ted官方網站上上傳的Ted talks影片資訊，包含講題、講者姓名、上傳時間、評論等內容，請大家完成以下任務。

1. 繪製出每一年的影片數量長條圖，以影片時間(filmdate)為準
2. 挑選dataset中2012年的所有資料，並列出出現次數前五多的tag (請將TEDx這個tag忽略不計)
3. 計算有這六個tag的影片與沒有這些tag的影片(不限於2012年)的評論(comments)數量平均，請問兩類影片的平均評論數量是多少?

### Tips
    與上題相同，你可以先將filmdate轉為pd.datetime格式，再擷取年份的資料
    tag欄位紀錄的內容都是文字而非列表形式，你可能需要用到python針對字串處理的功能如split或是strip幫助你清理資料
    除了python內建的字串處理函數外，也可以考慮使用pandas.str內使用的函數(ex. pandas.str.contains)

[source](https://www.kaggle.com/rounakbanik/ted-talks)

In [ ]:
df_ted = pd.read_csv('dataset/ted.csv')

In [ ]:
# 將published_date轉換為datetime格式，再取出年份的資料
df_ted['year'] = pd.to_datetime(df_ted.published_date, unit='s').apply(lambda x: x.year)

In [ ]:
# 呈現每個年份的資料數量
df_ted['year'].value_counts().plot.bar()

In [ ]:
# 取出年份為2012年的資料內的tag欄位，並且使用迴圈的方式將每個tag文字的內容做解析、再存入一個新的list中
t_list = []
for t in df_ted[df_ted['year'] == 2012].tags:
    t_list.extend(t.strip('[]\'').split('\', \''))

In [ ]:
# 呈現出現次數第二到第六名的關鍵詞
pd.Series(t_list).value_counts().iloc[1:6]

In [ ]:
# 擷取popular_tags，再創造一個新的欄位顯示這筆資料是否有popular_tags
popular_tags = pd.Series(t_list).value_counts().iloc[1:6]

df_ted['popular_tag'] = df_ted.tags.str.contains('|'.join(popular_tags.index))

In [ ]:
# 根據popular_tag的有無分別計算資料內的評論數量平均
df_ted.groupby('popular_tag')['comments'].mean()

---
除了直接觀察每篇文章的評論次數之外，我們也擔心每年TEDx的基本曝光率跟觀看人數會對我們的評估造成影響，為了驗證這個可能性，請大家做以下的幾件事情

1. 先將文章評論數量以log做轉換，再繪製每一年log(文章評論數量)的箱形圖，觀察每年的文章評論分布是否有差異
2. 為了校正每年的差異，請把每筆資料的comments數量減去年平均，並且將結果新增為comments_correction欄位 (例如: 2015年的資料評論數量平均為118，那每一個在2015年的TEDx影片評論數量都需要減去118)
3. 再一次計算有這六個tag的影片與沒有這些tag的影片在comments_correction的平均，並且觀察是否還有差異呢

## Tips
    可以使用np.log做log的轉換
    在最簡單的方法上，使用groupby方法計算各年平均，再用merge將年平均的資料與原本資料合併，就可以對齊評論與評論年平均的資料

In [ ]:
# 使用np.log 方法將comments數值取log
df_ted['log_comments'] = np.log(df_ted.comments)

# 繪製log_comments 在不同年份的箱形圖
df_ted.boxplot('log_comments', by='year')

In [ ]:
# 計算每年的平均comments數量，並且將欄位名稱更改為year_comments
year_comments = df_ted.groupby('year', as_index=False).comments.mean().rename({'comments':'year_comments'}, axis=1)

In [ ]:
# 使用merge的方法把年平均與原本的資料依照year的欄位合併起來
df_ted = df_ted.merge(year_comments, on='year', how='left')

# 計算每筆資料評論數量減去評論的每年平均，並儲存成comments_correction欄位
df_ted['comments_correction'] = df_ted['comments'] - df_ted['year_comments']

In [ ]:
# 根據popular_tag的有無分別計算資料內comments_correction的平均
df_ted.groupby('popular_tag')['comments_correction'].mean()